In [ ]:
# installing required packages for this notebook session
sc.install_pypi_package("haversine")

In [ ]:
from haversine import haversine, Unit
import pyspark.sql.functions as F

def get_distance(lat1,lon1,lat2,lon2):
    coord_1 = (lat1,lon1)
    coord_2 = (lat2,lon2)

    distance = haversine(coord_1,coord_2,unit=Unit.METERS)
    
    return distance
    
get_distance_udf = F.udf(get_distance)

In [ ]:
day = 20
df = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/16-graph-joined-minute/MO_1510{day}/")
df.filter("id_avl_1 != id_avl_2").write.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/17-graphs-minutes-no-repeated/MO_1510{day}/")


In [ ]:
day = 20
df_no_repeated = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/17-graphs-minutes-no-repeated/MO_1510{day}/")

df_final = df_no_repeated.withColumn("distance",get_distance_udf(F.col("latitude_1"),F.col("longitude_1"),F.col("latitude_2"),F.col("longitude_2")))

df_final = df_final.repartition("graph_id")

df_final.filter("distance <= 100")\
    .repartition("graph_id")\
    .write.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/18-graphs-minutes-within-100m/MO_1510{day}/")

In [ ]:
distances = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/18-graphs-minutes-within-100m/MO_15101/")
df_counts = distances.groupby("id_avl_1","graph_id").agg(F.countDistinct("id_avl_2").alias("counter_buses_connections"))

df_counts.agg(F.avg("counter_buses_connections")).show()
 
x = df_counts.approxQuantile("counter_buses_connections", [0.25,0.5,0.75], 0.0001)